# 加载基本库

In [8]:
import pandas as pd
import datetime
import ffm
import time
# from sklearn import ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
# import lightgbm as lgb
from sklearn.metrics import log_loss
import xgboost
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)
import sklearn
sklearn.__version__

/Users/yuhua/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


'0.19.0'

# 加载数据项

In [9]:
train_df = pd.read_table('/Users/yuhua/先验CTR模型线上观察/Alimama比赛/round1_ijcai_18_train_20180301.txt',sep=' ')
test_df = pd.read_table('/Users/yuhua/先验CTR模型线上观察/Alimama比赛/round1_ijcai_18_test_a_20180301.txt',sep=' ')
train_df.columns

Index([u'instance_id', u'item_id', u'item_category_list',
       u'item_property_list', u'item_brand_id', u'item_city_id',
       u'item_price_level', u'item_sales_level', u'item_collected_level',
       u'item_pv_level', u'user_id', u'user_gender_id', u'user_age_level',
       u'user_occupation_id', u'user_star_level', u'context_id',
       u'context_timestamp', u'context_page_id', u'predict_category_property',
       u'shop_id', u'shop_review_num_level', u'shop_review_positive_rate',
       u'shop_star_level', u'shop_score_service', u'shop_score_delivery',
       u'shop_score_description', u'is_trade'],
      dtype='object')

# 预处理数据

In [10]:
%run FeatureProcess.py
featProc = FeatureProcess(   target="is_trade", 

                            categorical=[  
                                            'shop_id',
                                            'item_brand_id',
                                            'item_city_id',
                                            'item_price_level',
                                            'item_sales_level',
                                            'item_collected_level',
                                            'item_pv_level',
                                            'user_gender_id',
                                            'user_age_level',
                                            'user_occupation_id',
                                            'user_star_level',
                                            'context_page_id',
                                            'shop_review_num_level',
                                            'shop_star_level'], 

                            numerical=[     'shop_review_positive_rate',
                                            'shop_score_service',
                                            'shop_score_delivery',
                                            'shop_score_description']
                                )

#train_df = featProc.fillempty(train_df, -0.000000000001)
#df = featProc.toOneHot(train_df)

In [13]:
def train_and_test_xgboost(train_df, test_df):
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    xgb = xgboost.XGBClassifier(nthread=20)
    xgb.fit(X_train, y_train)
    
    return log_loss(y_test,xgb.predict_proba(X_test))

def train_and_test_lr(train_df, test_df):
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    lr = LogisticRegression()
    lr.fit(X_train, y_train)
    
    return log_loss(y_test,lr.predict_proba(X_test))

def train_and_test_randomforest(train_df, test_df):
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
#     rf = RandomForestClassifier(n_estimators=32, max_depth=40, min_samples_split=100, min_samples_leaf=10,  criterion='entropy',
#                         max_features=8, verbose = 1,  bootstrap=False, n_jobs=10)
#     RandomForestClassifier(criterion='entropy',n_estimators=100,n_jobs=15)

    rf = RandomForestClassifier(n_estimators=32, max_depth=20, min_samples_split=100, min_samples_leaf=10,  criterion='entropy', max_features=8, verbose = 1,  bootstrap=False)
    rf.fit(X_train, y_train)
    
    return log_loss(y_test,rf.predict_proba(X_test))

def train_and_test_gbdt(train_df, test_df):
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    gbdt = GradientBoostingClassifier()
    gbdt.fit(X_train, y_train)
    
    return log_loss(y_test,gbdt.predict_proba(X_test))

def train_and_test_gbdt(train_df, test_df):
    X_train = train_df.copy()
    del X_train['is_trade']
    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    gbdt = GradientBoostingClassifier()
    gbdt.fit(X_train, y_train)
    
    return log_loss(y_test,gbdt.predict_proba(X_test))

def train_and_test_ffm(train_df, test_df):
#     X_train = train_df.copy()
#     del X_train['is_trade']
#     y_train = train_df['is_trade']
    
#     X_test = test_df.copy()
#     del X_test['is_trade']
#     y_test = test_df['is_trade']
    
    
#     X_train = [tuple(x) for x in X_train.values]
#     y_train = [y for y in y_train.values]
    
    
    
#     print "train_df to FFM length =", len(train_df)
#     ffm_train = ffm.FFMData(featProc.cacheRun(featProc.toFFMData, train_df))
    
    print "test_df to FFM length =", len(test_df)
    ffm_test = ffm.FFMData(featProc.cacheRun(featProc.toFFMData, test_df))
    
#     print "start training ..."
#     model = ffm.FFM(eta=0.1, lam=0.0001, k=4)
#     model.init_model(ffm_train)

#     y_pred = model.predict_proba(ffm_test)
    
#     lls = log_loss(y_pred,gbdt.predict_proba(X_test))
#     print "lls =",lls
#     return lls
    return 0
    



In [ ]:
%run BaseFrame.py

# tat = BaseFrame(train_df, 0.05,'context_timestamp', True)
# print "xgboost=",tat.kflod_validation_seq(10,train_and_test_xgboost)

# tat = BaseFrame(train_df, 0.05,'context_timestamp', True)
# print "lr=",tat.kflod_validation_seq(10,train_and_test_lr)

# tat = BaseFrame(train_df, 0.05,'context_timestamp', True)
# print "randomforest=",tat.kflod_validation_seq(10,train_and_test_randomforest)

# tat = BaseFrame(train_df, 0.05,'context_timestamp', True)
# print "gbdt=",tat.kflod_validation_seq(10,train_and_test_gbdt)

tat = BaseFrame(train_df, 0.05,'context_timestamp', True)
print "ffm=",tat.kflod_validation_seq(10,train_and_test_ffm)


ffm= training and testing ...
Doing 0 1  train_len=45424 test_len=23907
test_df to FFM length = 23907
From Cache ...


In [2]:
# t1 = time.time()
# df = train_df[0:1002]
# df = featProc.fillempty(df, -0.000000000001)
# # print "LLLLLen=", len(featProc.cacheRun(featProc.toFFMData, df).columns)

# print len(featProc.cacheRun(featProc.toFFMData, df)[0])

# print time.time() - t1

# dir(train_df["item_id"])
import pickle,ffm
f = ffm.FFMData(pickle.load(open("d25f691b0a6dfaf67971b243ae450999-toFFMData.pickle")))